# load dask cluster with data
load a parquet dataset into a dask cluster

In [1]:
# nuclio: ignore
import nuclio

In [2]:
# Copyright 2018 Iguazio
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
import json
import numpy as np
import pandas as pd

import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

from typing import List, Optional

def load_dask(
    context: MLClientCtx,
    src_data: str,
    dask_key: str = "dask_key",
    inc_cols: Optional[List[str]] = None,
    index_cols: Optional[List[str]] = None,
    dask_persist: bool = True,
    refresh_data: bool = True,
    scheduler_key: str = "scheduler"
) -> None:
    """Load dataset into an existing dask cluster
    
    dask jobs define the dask client parameters at the job level, this method will raise an error if no client is detected.
    
    :param context:         the function context
    :param src_data:        url of the data file or partitioned dataset as either
                            artifact DataItem, string, or path object (similar to 
                            pandas read_csv)
    :param dask_key:        destination key of data on dask cluster and artifact store
    :param inc_cols:        include only these columns (very fast)
    :param index_cols:      list of index column names (can be a long-running process)
    :param dask_persist:    (True) should the data be persisted (through the `client.persist` op)
    :param refresh_data:    (False) if the dask_key already exists in the dask cluster, this will 
                            raise an Exception.  Set to True to replace the existing cluster data.
    :param scheduler_key:   (scheduler) the dask scheduler configuration, json also logged as an artifact
    """
    if hasattr(context, "dask_client"):
        dask_client = context.dask_client
    else:
        raise Exception("a dask client was not found in the execution context")
    
    src_data = str(src_data)
    if isinstance(src_data, str):
        if os.path.isdir(src_data) or src_data.endswith("pq") or src_data.endswith("parquet"):
            df = dd.read_parquet(src_data)
        elif src_data.endswith("csv"):
            df = dd.read_csv(src_data)

    if dask_persist:
        df = dask_client.persist(df)
        if dask_client.datasets and dask_key in dask_client.datasets:
            dask_client.unpublish_dataset(dask_key)
        dask_client.publish_dataset(df, name=dask_key)
    
    if context:
        context.dask_client = dask_client
        
    # share the scheduler, whether data is persisted or not
    filepath = os.path.join(context.artifact_path, scheduler_key+".json")
    dask_client.write_scheduler_file(filepath)
    
    # we don't use log_dataset here until it can take into account
    # dask origin and apply dask describe.
    context.log_artifact(scheduler_key, local_path=scheduler_key+".json")

In [3]:
# nuclio: end-code

### save

In [4]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dask", kind="dask", with_doc=True,
                      handler=load_dask)

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dask"
fn.spec.description = "load dask cluster with data"
fn.metadata.categories = ["fileutils", "distributed"]
fn.metadata.labels = {"author": "yjb"}
fn.spec.remote = True
fn.spec.replicas = 12 
fn.spec.max_replicas = 12
fn.spec.service_type = "NodePort"
fn.spec.image = "mlrun/ml-models:test"

fn.save()
fn.export("function.yaml")

[mlrun] 2020-04-20 18:56:58,600 saving function: load-dask, tag: latest
[mlrun] 2020-04-20 18:56:58,649 function spec saved to path: function.yaml


### test

In [8]:
from mlrun import import_function, mount_v3io, NewTask

func = import_function("hub://load_dask").apply(mount_v3io())

task_params = {
    "name":        "tasks load dask cluster with data",
    "params" : {
        "persist"      : True,
        "refresh_data" : True,
        "dask_key"     : "dask_key"}}

In [9]:
run = func.run(NewTask(**task_params), 
               handler=load_dask, 
               inputs={"src_data" : "/User/artifacts/iris.parquet" },
               artifact_path="/User/artifacts")

[mlrun] 2020-04-20 18:57:26,081 starting run tasks load dask cluster with data uid=317e2ad14f3148db9316571c6e81e507  -> http://mlrun-api:8080
[mlrun] 2020-04-20 18:57:26,168 trying dask client at: tcp://mlrun-load-dask-a35164cb-c.default-tenant:8786
[mlrun] 2020-04-20 18:57:26,179 using remote dask scheduler (mlrun-load-dask-a35164cb-c) at: tcp://mlrun-load-dask-a35164cb-c.default-tenant:8786


/conda/lib/python3.6/site-packages/distributed/client.py:1079: VersionMismatchWarning: Mismatched versions found

blosc
+-------------------------+---------+
|                         | version |
+-------------------------+---------+
| client                  | 1.7.0   |
| scheduler               | None    |
| tcp://10.200.0.52:36152 | None    |
| tcp://10.200.0.53:43530 | None    |
| tcp://10.200.0.54:45422 | None    |
| tcp://10.200.0.56:46759 | None    |
| tcp://10.200.0.57:40730 | None    |
| tcp://10.200.0.58:37485 | None    |
| tcp://10.200.0.59:42117 | None    |
| tcp://10.200.0.60:35135 | None    |
| tcp://10.200.0.61:34958 | None    |
| tcp://10.200.0.62:33011 | None    |
| tcp://10.200.0.63:41829 | None    |
| tcp://10.200.0.64:39561 | None    |
+-------------------------+---------+

cloudpickle
+-------------------------+---------+
|                         | version |
+-------------------------+---------+
| client                  | 1.1.1   |
| scheduler               | 1.3

[mlrun] 2020-04-20 18:57:26,250 log artifact scheduler at /User/artifacts/scheduler.json, size: None, db: Y

[mlrun] 2020-04-20 18:57:26,277 run ended with state 


/conda/lib/python3.6/site-packages/distributed/client.py:1079: VersionMismatchWarning: Mismatched versions found

blosc
+-------------------------+---------+
|                         | version |
+-------------------------+---------+
| client                  | 1.7.0   |
| scheduler               | None    |
| tcp://10.200.0.52:36152 | None    |
| tcp://10.200.0.53:43530 | None    |
| tcp://10.200.0.54:45422 | None    |
| tcp://10.200.0.56:46759 | None    |
| tcp://10.200.0.57:40730 | None    |
| tcp://10.200.0.58:37485 | None    |
| tcp://10.200.0.59:42117 | None    |
| tcp://10.200.0.60:35135 | None    |
| tcp://10.200.0.61:34958 | None    |
| tcp://10.200.0.62:33011 | None    |
| tcp://10.200.0.63:41829 | None    |
| tcp://10.200.0.64:39561 | None    |
+-------------------------+---------+

cloudpickle
+-------------------------+---------+
|                         | version |
+-------------------------+---------+
| client                  | 1.1.1   |
| scheduler               | 1.3

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
,...6e81e507,0,Apr 20 18:57:26,completed,tasks load dask cluster with data,v3io_user=iguaziokind=daskowner=iguaziohost=jupyter-iguazio-fbcf6f67b-bcshp,src_data,persist=Truerefresh_data=Truedask_key=dask_key,,scheduler


to track results use .show() or .logs() or in CLI: 
!mlrun get run 317e2ad14f3148db9316571c6e81e507  , !mlrun logs 317e2ad14f3148db9316571c6e81e507 
[mlrun] 2020-04-20 18:57:26,300 run executed, status=completed


In [10]:
func.status.to_dict()

{'scheduler_address': 'tcp://mlrun-load-dask-a35164cb-c.default-tenant:8786',
 'cluster_name': 'mlrun-load-dask-a35164cb-c',
 'node_ports': {'dashboard': 30738, 'scheduler': 30153}}

### our cluster

Let"s load the scheduler file into a cluster in this notebook:

In [11]:
import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

In [12]:
client = Client(func.status.to_dict()['scheduler_address'])
client

/conda/lib/python3.6/site-packages/distributed/client.py:1079: VersionMismatchWarning: Mismatched versions found

blosc
+-------------------------+---------+
|                         | version |
+-------------------------+---------+
| client                  | 1.7.0   |
| scheduler               | None    |
| tcp://10.200.0.52:36152 | None    |
| tcp://10.200.0.53:43530 | None    |
| tcp://10.200.0.54:45422 | None    |
| tcp://10.200.0.56:46759 | None    |
| tcp://10.200.0.57:40730 | None    |
| tcp://10.200.0.58:37485 | None    |
| tcp://10.200.0.59:42117 | None    |
| tcp://10.200.0.60:35135 | None    |
| tcp://10.200.0.61:34958 | None    |
| tcp://10.200.0.62:33011 | None    |
| tcp://10.200.0.63:41829 | None    |
| tcp://10.200.0.64:39561 | None    |
+-------------------------+---------+

cloudpickle
+-------------------------+---------+
|                         | version |
+-------------------------+---------+
| client                  | 1.1.1   |
| scheduler               | 1.3

Client Scheduler: tcp://mlrun-load-dask-a35164cb-c.default-tenant:8786 Dashboard: http://mlrun-load-dask-a35164cb-c.default-tenant:8787/status,Cluster Workers: 12 Cores: 12 Memory: 67.25 GB


In [13]:
list(client.list_datasets())

['dask_key']

In [14]:
client.datasets['dask_key']

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),labels
npartitions=1,,,,,
,float64,float64,float64,float64,float64
,...,...,...,...,...
